# Create triplets

In [ ]:
import numpy as np 
import pandas as pd 
import sqlite3
import os

In [ ]:
train = pd.read_csv('/kaggle/input/shopee-product-matching/train.csv')

In [ ]:
con = sqlite3.connect("data.sqlite")

In [ ]:
train['idx'] = [i for i in range(len(train))]
train.to_sql("train", con, if_exists="replace")

In [ ]:
pos_query ="""
SELECT a.image as anchor, b.image as positive, a.label_group
FROM train as a
LEFT JOIN train as b
ON a.label_group = b.label_group 
WHERE a.idx > b.idx AND a.image!=b.image AND a.image_phash != b.image_phash
""" 

pos_pairs = pd.read_sql_query(pos_query, con)
pos_pairs = pos_pairs.set_index('label_group')

In [ ]:

def sample_negative_pairs(pos_pairs,train):

    label_groups = pos_pairs.index.unique()
    
    pos_pairs['negative'] = np.nan
    
    for group in label_groups:
        candidates = train[train['label_group'] != group]['image']

            
        num_candidates = len(pos_pairs.loc[[group]].index)
        pos_pairs.loc[[group],'negative'] = np.random.choice(candidates,size=num_candidates)
        
    return pos_pairs.dropna()


In [ ]:
anchor_pos_neg = pd.concat([sample_negative_pairs(pos_pairs,train) for i in range(3)])

In [ ]:
anchor_pos_neg.to_csv('anchor_pos_neg.csv',index=False)

In [ ]:
anchor_pos_neg.head()